In [3]:
from brt.router.utils import generate_dst_indices
import torch

dst_num = 4
gates = torch.randn((4, 4)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

route_indices = (
    torch.zeros(
        gates.size(0),
        dst_num,
        dtype=torch.int64,
        device=gates.device,
    )
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
supported_capacities = None

local_indices, dst_loads = generate_dst_indices(route_indices, supported_capacities)
print(local_indices)
print(dst_loads)


tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [2, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([2, 1, 1, 0], device='cuda:0', dtype=torch.int32)


In [4]:
from brt._C.router import dispatch_with_dst_indices_1d
import torch

in_data = torch.randn(4, 4, 4, dtype=torch.float32, requires_grad=True).cuda()

total_load = torch.sum(dst_loads).item()
out_data = torch.zeros((total_load, 4), dtype=in_data.dtype).cuda()

new_gates = torch.ones_like(gates, dtype=torch.float32).cuda()

print(in_data)
print(local_indices)
print(dst_loads)
out_data = dispatch_with_dst_indices_1d(
    in_data,
    local_indices,
    dst_loads,
)

print(out_data)


tensor([[[-0.9770, -1.1473, -1.0334, -0.8232],
         [-0.5851, -1.3774,  1.2046, -1.2152],
         [-1.0122, -0.6246,  1.0016,  1.6380],
         [ 1.9609,  0.6950, -1.5311,  1.5943]],

        [[-1.3046,  1.0876,  0.6222,  0.4509],
         [-1.3509,  0.0613, -0.7699, -0.5236],
         [ 0.0943,  0.9361,  0.5199,  0.4449],
         [-0.9209,  0.1338,  0.9276, -1.0284]],

        [[ 0.0651, -0.5160,  1.7939, -1.1250],
         [ 0.7448,  0.4920, -1.0126,  2.8997],
         [ 1.5567,  0.9394, -0.1925,  0.1449],
         [-1.5481,  0.1049,  1.3766, -1.9229]],

        [[ 0.0229,  0.1393, -0.0357,  0.1752],
         [-1.0998,  0.9090, -1.3338, -0.3332],
         [ 0.3678,  0.6949, -0.8650, -0.0833],
         [-0.6429, -0.0560, -0.3000, -0.8104]]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)
tensor([[0, 1, 0, 0],
        [0, 0, 1, 0],
        [1, 0, 0, 0],
        [2, 0, 0, 0]], device='cuda:0', dtype=torch.int32)
tensor([2, 1, 1, 0], device='cuda:0', dtype=torch.int32)
tensor(

In [5]:
from brt._C.router import combine_with_src_indices

final_data = combine_with_src_indices(out_data, local_indices, dst_loads)

print(final_data)
print(torch.allclose(final_data, in_data))
print(final_data.shape)

tensor([[[-0.9770, -1.1473, -1.0334, -0.8232],
         [-0.5851, -1.3774,  1.2046, -1.2152],
         [-1.0122, -0.6246,  1.0016,  1.6380],
         [ 1.9609,  0.6950, -1.5311,  1.5943]],

        [[-1.3046,  1.0876,  0.6222,  0.4509],
         [-1.3509,  0.0613, -0.7699, -0.5236],
         [ 0.0943,  0.9361,  0.5199,  0.4449],
         [-0.9209,  0.1338,  0.9276, -1.0284]],

        [[ 0.0651, -0.5160,  1.7939, -1.1250],
         [ 0.7448,  0.4920, -1.0126,  2.8997],
         [ 1.5567,  0.9394, -0.1925,  0.1449],
         [-1.5481,  0.1049,  1.3766, -1.9229]],

        [[ 0.0229,  0.1393, -0.0357,  0.1752],
         [-1.0998,  0.9090, -1.3338, -0.3332],
         [ 0.3678,  0.6949, -0.8650, -0.0833],
         [-0.6429, -0.0560, -0.3000, -0.8104]]], device='cuda:0')
True
torch.Size([4, 4, 4])
